In [1]:
from urllib import request, parse, error
import sys

TARGET = 'http://crypto-class.appspot.com/po?er='
#--------------------------------------------------------------
# padding oracle
#--------------------------------------------------------------

def query(q):
    target = TARGET + parse.quote(q)    # Create query URL
    try:
        f = request.urlopen(target)          # Wait for response
        return True
    except error.HTTPError as e:          
#         print("We got: {}".format(e.code))       # Print response code
        if e.code == 404:
            return True # good padding
        return False # bad padding


In [2]:
query('f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4')

True

In [3]:
query('f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd')

False

In [4]:
def xor(a, b):
    return [a[i] ^ b[i] for i in range(len(a))]

def str_hex(s):
    return [b for b in bytes.fromhex(s)]

def num_str(arr):
    return ''.join('{:02x}'.format(b) for b in arr)

In [5]:
def decrypt(pre, curr, after):
    known = []
    pad = 0x01

    while pad <= 0x10:
        guess = 0
        pre_block = pre
        curr_block = str_hex(curr)
        after_block = after

        guess_pre = num_str(curr_block[:-(len(known)+1)])
        guess_post = '' if len(known) == 0 else curr_block[(-len(known)):]
        guess_post = xor(guess_post, known)
        guess_post = xor(guess_post, [pad] * len(known))
        guess_post = num_str(guess_post)
        curr_byte = curr_block[-(len(known)+1)]
        candidate = -1
        while guess <= 0xff:
            guess_curr = curr_byte ^ guess ^ pad
            guess_curr = num_str([guess_curr])
            query_str = pre_block + guess_pre + guess_curr + guess_post + after_block
            if query(query_str):
                candidate = max(candidate, guess)
                if candidate > 0x10:
                    break
            guess += 1
        if candidate == -1:
            raise Exception('Failed to hack the block')
        known.insert(0, candidate)
        pad += 1
        print('Decrypted {} bytes: {}'.format(len(known), known))
    return ''.join([chr(x) for x in known])

In [6]:
cipher = 'f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4'
blocks = [''.join([cipher[i * 32 + j] for j in range(32)]) for i in range(len(cipher) // 32)]

In [7]:
res = ''
for i in range(0, len(blocks)-1):
    curr = decrypt(''.join(blocks[:i]), blocks[i], blocks[i+1] if i < len(blocks)-1 else '')
    print('Decrypted block {}: {}'.format(i, curr))
    res += curr
print(res)

Decrypted 1 bytes: [32]
Decrypted 2 bytes: [115, 32]
Decrypted 3 bytes: [100, 115, 32]
Decrypted 4 bytes: [114, 100, 115, 32]
Decrypted 5 bytes: [111, 114, 100, 115, 32]
Decrypted 6 bytes: [87, 111, 114, 100, 115, 32]
Decrypted 7 bytes: [32, 87, 111, 114, 100, 115, 32]
Decrypted 8 bytes: [99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 9 bytes: [105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 10 bytes: [103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 11 bytes: [97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 12 bytes: [77, 97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 13 bytes: [32, 77, 97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 14 bytes: [101, 32, 77, 97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 15 bytes: [104, 101, 32, 77, 97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted 16 bytes: [84, 104, 101, 32, 77, 97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32]
Decrypted block 0: The Magic Words 
Decrypted 1 bytes: 

In [8]:
res

'The Magic Words are Squeamish Ossifrage\t\t\t\t\t\t\t\t\t'